Model utilized: Albert fine tuned

https://huggingface.co/textattack/albert-base-v2-yelp-polarity

Trained with dataset of Yelp from 2015 with labels: 

https://huggingface.co/datasets/yelp_polarity

### Installing libraries

In [1]:
!pip install pytorch-ignite transformers parallelformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.1/264.1 kB 30.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 17.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for parallelformers: filename=parallelformers-1.2.7-py3-none-any.whl size=117772 sha256=74b4a954c7aae2ca4f9d5bcdccaf2b868f34174091b48fe8e52896943c9fe2d7
  Stored in directory: /root/.cache/pip/wheels/4f/19/42/8d74380c84a1e93401ee163f3bd545853051a4d895fd95ca2e
Successfully built parallelformers


### Load Dataset

In [2]:
!pip install gdown
!mkdir data
!cd data
!gdown 1QhtF1UAbVMJyAcqVJDucYwmib0hHCP1A
!mv 'Dataset Yelp.zip' data.zip
!unzip data.zip
!mv  -v /notebooks/data/Dataset\ Yelp/* /notebooks/data/
!rm -rf /notebooks/data/Dataset\ Yelp

Downloading...
From: https://drive.google.com/uc?id=1QhtF1UAbVMJyAcqVJDucYwmib0hHCP1A
To: /notebooks/Dataset Yelp.zip
100%|██████████████████████████████████████| 4.23G/4.23G [01:05<00:00, 64.4MB/s]
Archive:  data.zip
   creating: Dataset Yelp/
  inflating: Dataset Yelp/business.json  
  inflating: Dataset Yelp/checkin.json  
  inflating: Dataset Yelp/review.json  
  inflating: Dataset Yelp/tip.json   
  inflating: Dataset Yelp/user.json  
mv: cannot stat '/notebooks/data/Dataset Yelp/*': No such file or directory


### Checking GPU availability for Paperspace instance

In [3]:
#### CHECKING CUDA SUPPORT AND VERSION
#import torch
#torch.cuda.is_available()
#!nvcc --version
#!nvidia-smi -q
#!nvidia-smi --list-gpus

### Loading and preparing the dataset

In [2]:
from datasets import load_dataset
dataset = load_dataset("json", data_files="./data/review.json")
dataset = dataset['train'].remove_columns(["user_id", "business_id", "date", "useful", "funny", "cool", "stars"])

Using custom data configuration default-47ba29970f60c9e7


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-47ba29970f60c9e7/0.0.0/da492aad5680612e4028e7f6ddc04b1dfcec4b64db470ed7cc5f2bb265b9b6b5. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [3]:
dataset_1 = dataset.select(range(1000000))
dataset_2 = dataset.select(range(1000000, 2000000))
dataset_3 = dataset.select(range(2000000, 3000000))
dataset_4 = dataset.select(range(3000000, 4000000))
dataset_5 = dataset.select(range(4000000, 5000000))
dataset_6 = dataset.select(range(5000000, 6000000))
dataset_7 = dataset.select(range(6000000, 6990280))

### Preparing batch pipeline in 7 pieces

In [4]:
from transformers import pipeline
classifier = pipeline("sentiment-analysis", model="textattack/albert-base-v2-yelp-polarity", device=0)

Downloading:   0%|          | 0.00/736 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/44.6M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/742k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/156 [00:00<?, ?B/s]

In [5]:
def add_predictions(records):

    predictions = classifier([record for record in records['text']], truncation=True)

    if isinstance(predictions, list):
        return {"labels": [pred["label"] for pred in predictions], "scores": [pred["score"] for pred in predictions]}
    else:
        return {"labels": predictions["label"], "scores": predictions["score"]}

In [9]:
dataset_predicted_1 = dataset_1.map(add_predictions, batched=True, batch_size=64)


Parameter 'function'=<function add_predictions at 0x7fc0dbe01670> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/15625 [00:00<?, ?ba/s]

/usr/local/lib/python3.9/dist-packages/transformers/pipelines/base.py:1012: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


FileNotFoundError: [Errno 2] No such file or directory: './data/datacsv/dataset_1.csv'

In [11]:
dataset_predicted_1.remove_columns(["text"]).to_csv("./data/dataset_1.csv")

Creating CSV from Arrow format:   0%|          | 0/100 [00:00<?, ?ba/s]

54764117

In [12]:
dataset_predicted_2 = dataset_2.map(add_predictions, batched=True, batch_size=64)
dataset_predicted_2.remove_columns(["text"]).to_csv("./data/dataset_2.csv")

  0%|          | 0/15625 [00:00<?, ?ba/s]

/usr/local/lib/python3.9/dist-packages/transformers/pipelines/base.py:1012: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Creating CSV from Arrow format:   0%|          | 0/100 [00:00<?, ?ba/s]

54764262

In [5]:
dataset_predicted_3 = dataset_3.map(add_predictions, batched=True, batch_size=64)
dataset_predicted_3.remove_columns(["text"]).to_csv("./data/dataset_3.csv")

Parameter 'function'=<function add_predictions at 0x7fa3bc69d9d0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/15625 [00:00<?, ?ba/s]

/usr/local/lib/python3.9/dist-packages/transformers/pipelines/base.py:1012: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Creating CSV from Arrow format:   0%|          | 0/100 [00:00<?, ?ba/s]

54764193

In [6]:
dataset_predicted_4 = dataset_4.map(add_predictions, batched=True, batch_size=64)
dataset_predicted_4.remove_columns(["text"]).to_csv("./data/dataset_4.csv")

Parameter 'function'=<function add_predictions at 0x7f69b85a64c0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/15625 [00:00<?, ?ba/s]

/usr/local/lib/python3.9/dist-packages/transformers/pipelines/base.py:1012: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Creating CSV from Arrow format:   0%|          | 0/100 [00:00<?, ?ba/s]

54763259

In [7]:
dataset_predicted_5 = dataset_5.map(add_predictions, batched=True, batch_size=64)
dataset_predicted_5.remove_columns(["text"]).to_csv("./data/dataset_5.csv")

  0%|          | 0/15625 [00:00<?, ?ba/s]

/usr/local/lib/python3.9/dist-packages/transformers/pipelines/base.py:1012: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Creating CSV from Arrow format:   0%|          | 0/100 [00:00<?, ?ba/s]

54764566

In [8]:
dataset_predicted_6 = dataset_6.map(add_predictions, batched=True, batch_size=64)
dataset_predicted_6.remove_columns(["text"]).to_csv("./data/dataset_6.csv")

  0%|          | 0/15625 [00:00<?, ?ba/s]

/usr/local/lib/python3.9/dist-packages/transformers/pipelines/base.py:1012: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Creating CSV from Arrow format:   0%|          | 0/100 [00:00<?, ?ba/s]

54763692

In [6]:
dataset_predicted_7 = dataset_7.map(add_predictions, batched=True, batch_size=64)
dataset_predicted_7.remove_columns(["text"]).to_csv("./data/dataset_7.csv")

Parameter 'function'=<function add_predictions at 0x7fdda9bac790> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/15474 [00:00<?, ?ba/s]

/usr/local/lib/python3.9/dist-packages/transformers/pipelines/base.py:1012: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Creating CSV from Arrow format:   0%|          | 0/100 [00:00<?, ?ba/s]

54231364